In [2]:
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, Dataset
import pandas as pd

# Function for load data
def load_data(path,batch_size = 32):
    df = pd.read_csv(path)
#     df = df.head(10)
    df.columns = ['input_word','target_word']
    
    # Define maximum sequence lengths for letters
    max_input_len = max(len(word) for word in df['input_word'])
    max_target_len = max(len(word) for word in df['target_word'])

    # Define vocabulary mappings for letters
    input_letter_vocab = {'<pad>': 0, '<sos>': 1, '<eos>': 2}  # Add special tokens
    target_letter_vocab = {'<pad>': 0, '<sos>': 1, '<eos>': 2}  # Add special tokens
    letter_idx = 3

    input_str = ''
    target_str = ''
    # Preprocess the data and update vocabulary mappings for letters
    for input_word, target_word in zip(df['input_word'], df['target_word']):
        input_str += input_word
        target_str += target_word


    # Update vocabulary mappings for input letters
    for letter in sorted(set(input_str)):
        input_letter_vocab[letter] = letter_idx
        letter_idx += 1
    letter_idx = 3
    # Update vocabulary mappings for target letters
    for letter in sorted(set(target_str)):
        if letter not in target_letter_vocab:
            target_letter_vocab[letter] = letter_idx
            letter_idx += 1

    # Tokenize function at the letter level
    def tokenize_input_letters(word, vocab, max_len):
        token_ids = [vocab[char] for char in word if char in vocab]
        padded = token_ids[:max_len] + [vocab['<pad>']] * (max_len - len(token_ids))
        return torch.tensor(padded)

    def tokenize_target_letters(word, vocab, max_len):
        token_ids = [vocab[char] for char in word if char in vocab]
        padded =  [vocab['<pad>']]+ token_ids[:max_len] +[vocab['<pad>']] * (max_len - len(token_ids))
        return torch.tensor(padded)
    
    # Custom Dataset class for letter-level tokenization
    class CustomDataset(Dataset):
        def __init__(self, input_data, target_data, input_vocab, target_vocab, max_input_len, max_target_len):
            self.input_data = input_data
            self.target_data = target_data
            self.input_vocab = input_vocab
            self.target_vocab = target_vocab
            self.max_input_len = max_input_len
            self.max_target_len = max_target_len

        def __len__(self):
            return len(self.input_data)

        def __getitem__(self, idx):
            input_word = self.input_data[idx]
            target_word = self.target_data[idx]

            # Tokenize input and target words at the letter level
            input_letters = tokenize_input_letters(input_word, self.input_vocab, self.max_input_len)
            target_letters = tokenize_target_letters(target_word, self.target_vocab, self.max_target_len)

            return input_letters, target_letters

    # Create DataLoader
    custom_dataset = CustomDataset(df['input_word'], df['target_word'], input_letter_vocab, target_letter_vocab, max_input_len, max_target_len)
    data_loader1 = DataLoader(custom_dataset, batch_size=batch_size, shuffle = False )
    
    return custom_dataset,data_loader1, input_letter_vocab, target_letter_vocab, max_input_len, max_target_len



In [3]:
path1 = '/kaggle/input/aksharantar-sampled-dataset/aksharantar_sampled/ben/ben_train.csv'
custom_dataset1,train_loader_ben,a,b,_,_ = load_data(path1,batch_size = 64)
path2 = '/kaggle/input/aksharantar-sampled-dataset/aksharantar_sampled/ben/ben_valid.csv'
custom_dataset,val_loader_ben,_,_,_,_ = load_data(path2,batch_size = 64)
print(a,b)

{'<pad>': 0, '<sos>': 1, '<eos>': 2, 'a': 3, 'b': 4, 'c': 5, 'd': 6, 'e': 7, 'f': 8, 'g': 9, 'h': 10, 'i': 11, 'j': 12, 'k': 13, 'l': 14, 'm': 15, 'n': 16, 'o': 17, 'p': 18, 'q': 19, 'r': 20, 's': 21, 't': 22, 'u': 23, 'v': 24, 'w': 25, 'x': 26, 'y': 27, 'z': 28} {'<pad>': 0, '<sos>': 1, '<eos>': 2, 'ঁ': 3, 'ং': 4, 'ঃ': 5, 'অ': 6, 'আ': 7, 'ই': 8, 'ঈ': 9, 'উ': 10, 'ঊ': 11, 'ঋ': 12, 'এ': 13, 'ঐ': 14, 'ও': 15, 'ঔ': 16, 'ক': 17, 'খ': 18, 'গ': 19, 'ঘ': 20, 'ঙ': 21, 'চ': 22, 'ছ': 23, 'জ': 24, 'ঝ': 25, 'ঞ': 26, 'ট': 27, 'ঠ': 28, 'ড': 29, 'ঢ': 30, 'ণ': 31, 'ত': 32, 'থ': 33, 'দ': 34, 'ধ': 35, 'ন': 36, 'প': 37, 'ফ': 38, 'ব': 39, 'ভ': 40, 'ম': 41, 'য': 42, 'র': 43, 'ল': 44, 'শ': 45, 'ষ': 46, 'স': 47, 'হ': 48, '়': 49, 'া': 50, 'ি': 51, 'ী': 52, 'ু': 53, 'ূ': 54, 'ৃ': 55, 'ে': 56, 'ৈ': 57, 'ো': 58, 'ৌ': 59, '্': 60, 'ৎ': 61, '২': 62}


# **Attention Model**

In [4]:
import torch
import torch.nn as nn
import torch.optim as optim
from tqdm import tqdm

# Encoder class for attention model
class Encoder(nn.Module):
    def __init__(self, input_size, hidden_size, embed_size, encoder_layers=1, drop_prob=0.5, cell_type='gru', bidirectional=False):
        super(Encoder, self).__init__()
        self.hidden_size = hidden_size
        self.encoder_layers = encoder_layers
        self.cell_type = cell_type
        self.bidirectional = bidirectional
        self.dropout = nn.Dropout(drop_prob)
        self.embedding = nn.Embedding(input_size, embed_size)
        
        if cell_type == 'lstm':
            self.rnn = nn.LSTM(embed_size, hidden_size, encoder_layers, dropout=drop_prob, bidirectional=bidirectional, batch_first=True)
        elif cell_type == 'gru':
            self.rnn = nn.GRU(embed_size, hidden_size, encoder_layers, dropout=drop_prob, bidirectional=bidirectional, batch_first=True)
        else:
            self.rnn = nn.RNN(embed_size, hidden_size, encoder_layers, dropout=drop_prob, bidirectional=bidirectional, batch_first=True)
    
    def forward(self, x):
        embedded = self.dropout(self.embedding(x))
        output, hidden = self.rnn(embedded)
        
        if self.cell_type == 'lstm':
            hidden_states, cell_states = hidden
            if self.bidirectional:
                hidden = (torch.cat([hidden_states[-2], hidden_states[-1]], dim=1).unsqueeze(0), 
                          torch.cat([cell_states[-2], cell_states[-1]], dim=1).unsqueeze(0))
            else:
                hidden = (hidden_states[-1].unsqueeze(0), cell_states[-1].unsqueeze(0))
        else:
            if self.bidirectional:
                hidden = torch.cat([hidden[-2], hidden[-1]], dim=1).unsqueeze(0)
            else:
                hidden = hidden[-1].unsqueeze(0)

        return output, hidden
    
# Attention class
class Attention(nn.Module):
    def __init__(self, hidden_size):
        super(Attention, self).__init__()
        self.attention = nn.Linear(hidden_size * 2, hidden_size)
        self.v = nn.Parameter(torch.rand(hidden_size))

    def forward(self, hidden, encoder_outputs):
        seq_len = encoder_outputs.shape[1]
        hidden = hidden.repeat(seq_len, 1, 1).transpose(0, 1)
        
        energy = torch.tanh(self.attention(torch.cat((hidden, encoder_outputs), dim=2)))
        energy = energy.transpose(1, 2)
        v = self.v.repeat(encoder_outputs.size(0), 1).unsqueeze(1)
        
        attention_weights = torch.bmm(v, energy)
        return torch.softmax(attention_weights.squeeze(1), dim=1)

# Decoder class with attention
class DecoderWithAttention(nn.Module):
    def __init__(self, hidden_size, embed_size, output_size, attention, decoder_layers=1, drop_prob=0.5, cell_type='gru'):
        super(DecoderWithAttention, self).__init__()
        self.hidden_size = hidden_size
        self.decoder_layers = decoder_layers
        self.cell_type = cell_type
        self.attention = attention
        self.dropout = nn.Dropout(drop_prob)
        self.embedding = nn.Embedding(output_size, embed_size)
        
        input_size = hidden_size + embed_size
        if cell_type == 'lstm':
            self.rnn = nn.LSTM(input_size, hidden_size, decoder_layers, dropout=drop_prob, batch_first=True)
        elif cell_type == 'gru':
            self.rnn = nn.GRU(input_size, hidden_size, decoder_layers, dropout=drop_prob, batch_first=True)
        else:
            self.rnn = nn.RNN(input_size, hidden_size, decoder_layers, dropout=drop_prob, batch_first=True)
        
        self.fc = nn.Linear(hidden_size * 2, output_size)

    def forward(self, x, hidden, encoder_outputs):
        x = x.unsqueeze(1)
        embedded = self.dropout(self.embedding(x))
        
        attention_weights = self.attention(hidden[-1], encoder_outputs)
        context = attention_weights.unsqueeze(1).bmm(encoder_outputs)
        
        rnn_input = torch.cat((embedded, context), dim=2)
        
        if self.cell_type == 'lstm':
            output, (hidden, cell) = self.rnn(rnn_input, hidden)
        else:
            output, hidden = self.rnn(rnn_input, hidden)
        
        output = self.fc(torch.cat((output, context), dim=2).squeeze(1))
        
        if self.cell_type == 'lstm':
            return output, (hidden, cell)
        else:
            return output, hidden

# Seq2seq model
class Seq2Seq(nn.Module):
    def __init__(self, input_size, output_size, hidden_size, embed_size, encoder_layers=1, decoder_layers=1, drop_prob=0.3, cell_type='gru', bidirectional=True):
        super(Seq2Seq, self).__init__()
        self.encoder = Encoder(input_size, hidden_size, embed_size, encoder_layers, drop_prob, cell_type, bidirectional)
        self.attention = Attention(hidden_size * 2 if bidirectional else hidden_size)
        self.decoder = DecoderWithAttention(hidden_size * 2 if bidirectional else hidden_size, embed_size, output_size, self.attention, decoder_layers, drop_prob, cell_type)

    def forward(self, source, target, teacher_forcing_ratio=0.5):
        batch_size = source.size(0)
        target_len = target.size(1)
        output_vocab_size = self.decoder.embedding.num_embeddings

        outputs = torch.zeros(batch_size, target_len, output_vocab_size).to(source.device)

        encoder_outputs, encoder_hidden = self.encoder(source)
        decoder_hidden = self._init_decoder_hidden(encoder_hidden)
        decoder_input = target[:, 0]

        for t in range(1, target_len):
            decoder_output, decoder_hidden = self.decoder(decoder_input, decoder_hidden, encoder_outputs)
            outputs[:, t] = decoder_output
            teacher_force = torch.rand(1).item() < teacher_forcing_ratio
            t1 = decoder_output.argmax(1)
            decoder_input = target[:, t] if teacher_force else t1

        return outputs

    def _init_decoder_hidden(self, encoder_hidden):
        decoder_layers = self.decoder.decoder_layers
        if self.encoder.cell_type == 'lstm':
            encoder_hidden = (
                torch.cat([encoder_hidden[0][i] for i in range(encoder_hidden[0].shape[0])], dim=1).unsqueeze(0), 
                torch.cat([encoder_hidden[1][i] for i in range(encoder_hidden[1].shape[0])], dim=1).unsqueeze(0)
            )
            if encoder_hidden[0].shape[0] != decoder_layers:
                encoder_hidden = (
                    encoder_hidden[0][:decoder_layers],
                    encoder_hidden[1][:decoder_layers]
                )
        else:
            encoder_hidden = torch.cat([encoder_hidden[i] for i in range(encoder_hidden.shape[0])], dim=1).unsqueeze(0)
            if encoder_hidden.shape[0] != decoder_layers:
                encoder_hidden = encoder_hidden[:decoder_layers]

        return encoder_hidden


In [18]:
    
# # Training function
# def train(model, dataloader, criterion, optimizer, device):
#     model.train()
#     total_loss = 0
#     total_correct = 0
#     total_samples = 0
    
#     for latin, devanagari in tqdm(dataloader, desc='Training', unit='batch'):
#         latin = latin.to(device)
#         devanagari = devanagari.to(device)
        
#         optimizer.zero_grad()
        
#         output = model(latin, devanagari)
#         output_dim = output.shape[-1]
#         output1 = output.view(-1, output_dim)
#         devanagari1 = devanagari.view(-1)
        
#         loss = criterion(output1, devanagari1)
#         total_loss += loss.item()
        
#         loss.backward()
#         optimizer.step()
# #         break
        
#         max_values ,max_index = torch.max(output, 2) #output.argmax(dim=1)
#         mask = max_index > 9
#         max_index[mask] -= 2
# #             print(f"prediction:{max_index} '\n' actual:{devanagari}")
#         correct1=(max_index == devanagari).all(dim=1).sum().item()
# #             print(f"correct1:{correct1}")
# #         correct = (max_index == devanagari).sum().item()
# #             print(f"correct:{correct}")
#         total_correct += correct1
#         total_samples += devanagari.size(0)

#     accuracy = total_correct / total_samples
    
#     return model, total_loss / len(dataloader), accuracy

# # Evaluation function
# def evaluate(model, dataloader, criterion, device):
#     model.eval()
#     total_loss = 0
#     total_correct = 0
#     total_samples = 0
    
#     with torch.no_grad():
#         for latin, devanagari in tqdm(dataloader, desc='Evaluating', unit='batch'):
#             latin = latin.to(device)
#             devanagari = devanagari.to(device)
            
#             output = model(latin, devanagari)
# #                                             print("output:",output.shape)
#             output_dim = output.shape[-1]
# #                                                     print(f"output_dim {output_dim}")
# #             output = output.view(-1, output_dim)
# #             print("output.shape:",output.shape)
# #             print("output.:",output)
# #             print(f"devanagari: {devanagari}")
# #             devanagari = devanagari.view(-1)
# #             print(f"devanagari.view(-1): {devanagari}")
# #             
#             loss = criterion(output.view(-1, output_dim), devanagari.view(-1))
#             total_loss += loss.item()
            
#             # Calculate accuracy
#             max_values ,max_index = torch.max(output, 2) #output.argmax(dim=1)
#             mask = max_index > 9
#             max_index[mask] -= 2
# #             print(f"prediction:{max_index} '\n' actual:{devanagari}")
#             correct1=(max_index == devanagari).all(dim=1).sum().item()
# #             print(f"correct1:{correct1}")
#             correct = (max_index == devanagari).sum().item()
# #             print(f"correct:{correct}")
#             total_correct += correct1
#             total_samples += devanagari.size(0)
    
#     avg_loss = total_loss / len(dataloader)
#     accuracy = total_correct / total_samples
    
#     return avg_loss, accuracy*100

In [46]:
def train(model, dataloader, criterion, optimizer, device):
    model.train()
    total_loss = 0
    total_correct = 0
    total_samples = 0
    
    for latin, devanagari in tqdm(dataloader, desc='Training', unit='batch'):
        latin = latin.to(device)
        devanagari = devanagari.to(device)
        
        optimizer.zero_grad()
        
        output = model(latin, devanagari)
        output_dim = output.shape[-1]
        output = output.view(-1, output_dim)
        devanagari = devanagari.view(-1)
        
        loss = criterion(output, devanagari)
        total_loss += loss.item()
        
        loss.backward()
        optimizer.step()
        
        # Reshape the output and target to their original shape
        output = output.view(latin.size(0), -1, output_dim)
        devanagari = devanagari.view(latin.size(0), -1)
        
        max_index = output.argmax(dim=2)
        # Calculate word-level accuracy
        correct = (max_index == devanagari).all(dim=1).sum().item()
        total_correct += correct
        total_samples += devanagari.size(0)
    
    accuracy = total_correct / total_samples
    return model, total_loss / len(dataloader), accuracy*100


def evaluate(model, dataloader, criterion, device):
    model.eval()
    total_loss = 0
    total_correct = 0
    total_samples = 0
    
    with torch.no_grad():
        for latin, devanagari in tqdm(dataloader, desc='Evaluating', unit='batch'):
            latin = latin.to(device)
            devanagari = devanagari.to(device)
            
            output = model(latin, devanagari, teacher_forcing_ratio=0.0)
            output_dim = output.shape[-1]
            output = output.view(-1, output_dim)
            devanagari = devanagari.view(-1)
            
            loss = criterion(output, devanagari)
            total_loss += loss.item()
            
            # Reshape the output and target to their original shape
            output = output.view(latin.size(0), -1, output_dim)
            devanagari = devanagari.view(latin.size(0), -1)
            
            max_index = output.argmax(dim=2)
            mask = max_index > 9
            max_index[mask] -= 2            
            correct = (max_index == devanagari).all(dim=1).sum().item()  #  Calculate word-level accuracy
            total_correct += correct
            total_samples += devanagari.size(0)
    
    avg_loss = total_loss / len(dataloader)
    accuracy = total_correct / total_samples
    return avg_loss, accuracy * 100


In [8]:
# Example usage
input_size = 30  # Number of Latin characters
output_size = 70  # Number of Devanagari characters
embed_size = 16
hidden_size = 32
encoder_layers = 1
decoder_layers = 1
cell_type = 'rnn'
batch_size = 64
num_epochs = 12
drop_prob = 0.3
learning_rate = 0.001

# Assuming you have loaded your dataset into train_loader and val_loader

# Initialize the model, criterion, and optimizer
model = Seq2Seq(input_size, output_size, hidden_size,embed_size, encoder_layers,decoder_layers,drop_prob, cell_type)
print(model)

# model = Attention_model
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
ignore_index = 0
criterion = nn.CrossEntropyLoss(ignore_index = 0)
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

# Training loop
# for epoch in range(num_epochs):
#     trained_model, train_loss, train_acc  = train(model, train_loader_ben, criterion, optimizer, device)
#     val_loss, val_accuracy = evaluate(trained_model, val_loader_ben, criterion, device)
#     model = trained_model
#     print(f'Epoch {epoch+1}/{num_epochs}, Train Loss: {train_loss:.4f}, Train_acc: {train_acc},  Val Loss: {val_loss:.4f}, Val Accuracy: {val_accuracy:.4f}')

Seq2Seq(
  (encoder): Encoder(
    (dropout): Dropout(p=0.3, inplace=False)
    (embedding): Embedding(30, 16)
    (rnn): RNN(16, 32, batch_first=True, dropout=0.3, bidirectional=True)
  )
  (attention): Attention(
    (attention): Linear(in_features=128, out_features=64, bias=True)
  )
  (decoder): DecoderWithAttention(
    (attention): Attention(
      (attention): Linear(in_features=128, out_features=64, bias=True)
    )
    (dropout): Dropout(p=0.3, inplace=False)
    (embedding): Embedding(70, 16)
    (rnn): RNN(80, 64, batch_first=True, dropout=0.3)
    (fc): Linear(in_features=128, out_features=70, bias=True)
  )
)


/opt/conda/lib/python3.10/site-packages/torch/nn/modules/rnn.py:82: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.3 and num_layers=1
  warnings.warn("dropout option adds dropout after all but last "


In [41]:
# !pip install wandb
import wandb
import numpy as np
from types import SimpleNamespace
import random

In [42]:
wandb.login(key='bb3c7761be2856a8335d16d1483149380482ae9e')#bb3c7761be2856a8335d16d1483149380482ae9e


wandb: W&B API key is configured. Use `wandb login --relogin` to force relogin
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

In [9]:
sweep_config = {
    'method': 'bayes',
    'metric': {
      'name': 'val_accuracy',
      'goal': 'maximize'
    },
    'parameters': {
        'embedding_size':{
            'values': [16,32,64,128,256]
        },
        'dropout': {
            'values': [0.3, 0.2,0.5]
        },
        'encoder_layers': {
            'values': [1]
        },
        'decoder_layers':{
            'values': [1]
        },
        'hidden_layer_size':{
            'values': [16,32,64,128,256]
        },
        'cell_type': {
            'values': [ 'lstm','rnn', 'gru']
        },
        'bidirectional': {
            'values': [True, False]
        },
        'batch_size': {
            'values': [32,64]
        },
        'num_epochs': {
            'values': [10,12]
        },
        'learning_rate': {
            'values': [0.01,0.001]
        }
    }
}

sweep_id = wandb.sweep(sweep=sweep_config, project='DL_A3_Attention')


Create sweep with ID: vkbi461w
Sweep URL: https://wandb.ai/abanisingha1997/DL_A3_Attention/sweeps/vkbi461w


In [ ]:
def main():
    '''
    WandB calls main function each time with differnet combination.

    We can retrive the same and use the same values for our hypermeters.

    '''

    with wandb.init() as run:
        run_name="ct-"+str(wandb.config.cell_type)+"_el-"+str(wandb.config.encoder_layers)+"_dl-"+str(wandb.config.decoder_layers)+"_drop-"+str(wandb.config.dropout)+"_es-"+str(wandb.config.embedding_size)+"_hs-"+str(wandb.config.hidden_layer_size)+"_bs-"+str(wandb.config.batch_size)+"_ep-"+str(wandb.config.num_epochs)+"lr"+str(wandb.config.learning_rate)
        wandb.run.name=run_name

        
        model = Seq2Seq(input_size=30, output_size=70, hidden_size=wandb.config.hidden_layer_size,embed_size=wandb.config.embedding_size,encoder_layers=wandb.config.encoder_layers,
                        decoder_layers=wandb.config.decoder_layers,drop_prob=wandb.config.dropout, cell_type=wandb.config.cell_type, bidirectional=wandb.config.bidirectional)
        print(model)
        criterion = nn.CrossEntropyLoss()
        optimizer = optim.Adam(model.parameters(), lr=wandb.config.learning_rate)

        device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        model.to(device)
        path1 = '/kaggle/input/aksharantar-sampled-dataset/aksharantar_sampled/ben/ben_train.csv'
        custom_dataset1,train_loader_ben,a,b,_,_ = load_data(path1,batch_size = wandb.config.batch_size)
        path2 = '/kaggle/input/aksharantar-sampled-dataset/aksharantar_sampled/ben/ben_valid.csv'
        custom_dataset,val_loader_ben,_,_,_,_ = load_data(path2,batch_size = wandb.config.batch_size)

        # Training loop
        for epoch in range(wandb.config.num_epochs):
            trained_model, train_loss, train_acc = train(model, train_loader_ben, criterion, optimizer, device)
            val_loss, val_accuracy = evaluate(trained_model, val_loader_ben, criterion, device)
            model = trained_model
            wandb.log({'Epoch': epoch, 'train_loss': train_loss , ' val_loss': val_loss, 'val_accuracy':val_accuracy})
            print(f'Epoch {epoch+1}/{wandb.config.num_epochs}, Train Loss: {train_loss:.4f},Train_acc: {train_acc} Val Loss: {val_loss:.4f}, Val Accuracy: {val_accuracy:.4f}')

        
#         model_train(model,train,validation)
        
wandb.agent(sweep_id, function= main,count= 30) # calls main function for count number of times.
wandb.finish()

wandb: Agent Starting Run: p018rb83 with config:
wandb: 	batch_size: 64
wandb: 	bidirectional: False
wandb: 	cell_type: gru
wandb: 	decoder_layers: 1
wandb: 	dropout: 0.3
wandb: 	embedding_size: 64
wandb: 	encoder_layers: 1
wandb: 	hidden_layer_size: 32
wandb: 	learning_rate: 0.001
wandb: 	num_epochs: 10
wandb: Currently logged in as: abanisingha1997. Use `wandb login --relogin` to force relogin


/opt/conda/lib/python3.10/site-packages/torch/nn/modules/rnn.py:82: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.3 and num_layers=1
  warnings.warn("dropout option adds dropout after all but last "


Seq2Seq(
  (encoder): Encoder(
    (dropout): Dropout(p=0.3, inplace=False)
    (embedding): Embedding(30, 64)
    (rnn): GRU(64, 32, batch_first=True, dropout=0.3)
  )
  (attention): Attention(
    (attention): Linear(in_features=64, out_features=32, bias=True)
  )
  (decoder): DecoderWithAttention(
    (attention): Attention(
      (attention): Linear(in_features=64, out_features=32, bias=True)
    )
    (dropout): Dropout(p=0.3, inplace=False)
    (embedding): Embedding(70, 64)
    (rnn): GRU(96, 32, batch_first=True, dropout=0.3)
    (fc): Linear(in_features=64, out_features=70, bias=True)
  )
)


Evaluating: 100%|██████████| 64/64 [00:01<00:00, 61.40batch/s]


Epoch 1/10, Train Loss: 1.3245,Train_acc: 0.0 Val Loss: 2.0894, Val Accuracy: 0.0000


...

val_loss,▁▃▄▄▅▅▆▆▇▇▇█
Epoch,▁▂▂▃▄▄▅▅▆▇▇█
train_loss,█▃▂▂▂▂▁▁▁▁▁▁
val_accuracy,▁▄▅▆▇▇▇▇████
val_loss,4.61898
Epoch,11
train_loss,0.26714
val_accuracy,34.57875


wandb: Agent Starting Run: zbbie0q4 with config:
wandb: 	batch_size: 32
wandb: 	bidirectional: True
wandb: 	cell_type: lstm
wandb: 	decoder_layers: 1
wandb: 	dropout: 0.5
wandb: 	embedding_size: 64
wandb: 	encoder_layers: 1
wandb: 	hidden_layer_size: 256
wandb: 	learning_rate: 0.001
wandb: 	num_epochs: 12


/opt/conda/lib/python3.10/site-packages/torch/nn/modules/rnn.py:82: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.5 and num_layers=1
  warnings.warn("dropout option adds dropout after all but last "


Seq2Seq(
  (encoder): Encoder(
    (dropout): Dropout(p=0.5, inplace=False)
    (embedding): Embedding(30, 64)
    (rnn): LSTM(64, 256, batch_first=True, dropout=0.5, bidirectional=True)
  )
  (attention): Attention(
    (attention): Linear(in_features=1024, out_features=512, bias=True)
  )
  (decoder): DecoderWithAttention(
    (attention): Attention(
      (attention): Linear(in_features=1024, out_features=512, bias=True)
    )
    (dropout): Dropout(p=0.5, inplace=False)
    (embedding): Embedding(70, 64)
    (rnn): LSTM(576, 512, batch_first=True, dropout=0.5)
    (fc): Linear(in_features=1024, out_features=70, bias=True)
  )
)


Training:  35%|███▍      | 556/1600 [00:28<00:57, 18.15batch/s]

# **Best model**

In [32]:
# Best configutration
input_size = 30  # Number of Latin characters
output_size = 70  # Number of Devanagari characters
embed_size = 256
hidden_size = 256
encoder_layers = 1
decoder_layers = 1
cell_type = 'lstm'
batch_size = 64
num_epochs = 9
drop_prob = 0.3
learning_rate = 0.001
bidirectional=False

# Assuming you have loaded your dataset into train_loader and val_loader

# Initialize the model, criterion, and optimizer
Best_model = Seq2Seq(input_size, output_size, hidden_size,embed_size, encoder_layers,decoder_layers,drop_prob, cell_type, bidirectional)
print(Best_model)

# model = Attention_model
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
Best_model.to(device)
ignore_index = 0
criterion = nn.CrossEntropyLoss(ignore_index = 0)
optimizer = optim.Adam(Best_model.parameters(), lr=learning_rate)

Seq2Seq(
  (encoder): Encoder(
    (dropout): Dropout(p=0.3, inplace=False)
    (embedding): Embedding(30, 256)
    (rnn): LSTM(256, 256, batch_first=True, dropout=0.3)
  )
  (attention): Attention(
    (attention): Linear(in_features=512, out_features=256, bias=True)
  )
  (decoder): DecoderWithAttention(
    (attention): Attention(
      (attention): Linear(in_features=512, out_features=256, bias=True)
    )
    (dropout): Dropout(p=0.3, inplace=False)
    (embedding): Embedding(70, 256)
    (rnn): LSTM(512, 256, batch_first=True, dropout=0.3)
    (fc): Linear(in_features=512, out_features=70, bias=True)
  )
)


/opt/conda/lib/python3.10/site-packages/torch/nn/modules/rnn.py:82: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.3 and num_layers=1
  warnings.warn("dropout option adds dropout after all but last "


# **Prediction on test dataset for best model**

In [44]:
sweep_config = {
    'method': 'bayes',
    'metric': {
      'name': 'val_accuracy',
      'goal': 'maximize'
    },
    'parameters': {
        'embedding_size':{
            'values': [256]
        },
        'dropout': {
            'values': [0.3]
        },
        'encoder_layers': {
            'values': [1]
        },
        'decoder_layers':{
            'values': [1]
        },
        'hidden_layer_size':{
            'values': [256]
        },
        'cell_type': {
            'values': ['lstm']
        },
        'bidirectional': {
            'values': [True]
        },
        'batch_size': {
            'values': [64]
        },
        'num_epochs': {
            'values': [9]
        },
        'learning_rate': {
            'values': [0.001]
        }
    }
}

sweep_id = wandb.sweep(sweep=sweep_config, project='DL_A3_Attention')


Create sweep with ID: c1d3h9dz
Sweep URL: https://wandb.ai/abanisingha1997/DL_A3_Attention/sweeps/c1d3h9dz


In [62]:
def main():
    '''
    WandB calls main function each time with differnet combination.

    We can retrive the same and use the same values for our hypermeters.

    '''

    with wandb.init() as run:
        run_name="ct-"+str(wandb.config.cell_type)+"_el-"+str(wandb.config.encoder_layers)+"_dl-"+str(wandb.config.decoder_layers)+"_drop-"+str(wandb.config.dropout)+"_es-"+str(wandb.config.embedding_size)+"_hs-"+str(wandb.config.hidden_layer_size)+"_bs-"+str(wandb.config.batch_size)+"_ep-"+str(wandb.config.num_epochs)+"lr"+str(wandb.config.learning_rate)
        wandb.run.name=run_name

        
        model = Seq2Seq(input_size=30, output_size=70, hidden_size=wandb.config.hidden_layer_size,embed_size=wandb.config.embedding_size,encoder_layers=wandb.config.encoder_layers,
                        decoder_layers=wandb.config.decoder_layers,drop_prob=wandb.config.dropout, cell_type=wandb.config.cell_type, bidirectional=wandb.config.bidirectional)
        print(model)
        criterion = nn.CrossEntropyLoss()
        optimizer = optim.Adam(model.parameters(), lr=wandb.config.learning_rate)

        device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        model.to(device)
        path1 = '/kaggle/input/aksharantar-sampled-dataset/aksharantar_sampled/ben/ben_train.csv'
        custom_dataset1,train_loader_ben,a,b,_,_ = load_data(path1,batch_size = wandb.config.batch_size)
        
        path3 = '/kaggle/input/aksharantar-sampled-dataset/aksharantar_sampled/ben/ben_test.csv'
        c, test_loader_ben, input_vocab, target_vocab, max_length, _ = load_data(path3, batch_size=64)  # Use correct path3

        # Training loop
        for epoch in range(wandb.config.num_epochs):
            trained_model, train_loss, train_acc = train(model, train_loader_ben, criterion, optimizer, device)
            val_loss, val_accuracy = evaluate(trained_model, test_loader_ben, criterion, device)
            model = trained_model
            wandb.log({'Epoch': epoch, 'train_loss': train_loss , 'test_accuracy':val_accuracy})
            print(f'Epoch {epoch+1}/{wandb.config.num_epochs}, Train Loss: {train_loss:.4f}, test_accuracy: {val_accuracy:.4f}')

        
#         model_train(model,train,validation)
        
wandb.agent(sweep_id, function= main,count= 1) # calls main function for count number of times.
wandb.finish()

wandb: Agent Starting Run: pyjc8y6p with config:
wandb: 	batch_size: 64
wandb: 	bidirectional: True
wandb: 	cell_type: lstm
wandb: 	decoder_layers: 1
wandb: 	dropout: 0.3
wandb: 	embedding_size: 256
wandb: 	encoder_layers: 1
wandb: 	hidden_layer_size: 256
wandb: 	learning_rate: 0.001
wandb: 	num_epochs: 9


/opt/conda/lib/python3.10/site-packages/torch/nn/modules/rnn.py:82: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.3 and num_layers=1
  warnings.warn("dropout option adds dropout after all but last "


Seq2Seq(
  (encoder): Encoder(
    (dropout): Dropout(p=0.3, inplace=False)
    (embedding): Embedding(30, 256)
    (rnn): LSTM(256, 256, batch_first=True, dropout=0.3, bidirectional=True)
  )
  (attention): Attention(
    (attention): Linear(in_features=1024, out_features=512, bias=True)
  )
  (decoder): DecoderWithAttention(
    (attention): Attention(
      (attention): Linear(in_features=1024, out_features=512, bias=True)
    )
    (dropout): Dropout(p=0.3, inplace=False)
    (embedding): Embedding(70, 256)
    (rnn): LSTM(768, 512, batch_first=True, dropout=0.3)
    (fc): Linear(in_features=1024, out_features=70, bias=True)
  )
)


Evaluating: 100%|██████████| 64/64 [00:01<00:00, 54.40batch/s]


Epoch 1/9, Train Loss: 0.6319, test_accuracy: 18.8034


...

Evaluating: 100%|██████████| 64/64 [00:01<00:00, 54.03batch/s]


Epoch 8/9, Train Loss: 0.2752, test_accuracy: 32.8694


Evaluating: 100%|██████████| 64/64 [00:01<00:00, 54.47batch/s]


Epoch 9/9, Train Loss: 0.2659, test_accuracy: 32.3077


Epoch,▁▂▃▄▅▅▆▇█
test_accuracy,▁▄▆▆▆▇▇██
train_loss,█▃▃▂▂▂▁▁▁
Epoch,8
test_accuracy,32.30769
train_loss,0.26593


# **Prediction**

In [63]:
def decode_indices(indices, idx2token, target_vocab):
    valid_indices = []
    for idx in indices:
        if idx in idx2token and idx not in (target_vocab['<pad>'], target_vocab['<sos>'], target_vocab['<eos>']):
            valid_indices.append(idx)
#             print(valid_indices)
    decoded_text = ''
    for idx in valid_indices:
        decoded_text += idx2token[idx]
#         print(decoded_text)
    return decoded_text

def decode_indices_target(indices, idx2token, target_vocab):
    valid_indices = []
    for idx in indices:
        if idx in idx2token and idx not in (target_vocab['<pad>'], target_vocab['<sos>'], target_vocab['<eos>']):
            if idx < 10:
                valid_indices.append(idx)
            else:
                valid_indices.append(idx-3)
#             print(valid_indices)
    decoded_text = ''
    for idx in valid_indices:
        decoded_text += idx2token[idx]
#         print(decoded_text)
    return decoded_text

In [69]:
def decode_indices_target1(indices1,indices2, idx2token, target_vocab):
    valid_indices1 = []
    for idx in indices1:
        if idx in idx2token and idx not in (target_vocab['<pad>'], target_vocab['<sos>'], target_vocab['<eos>']):
            valid_indices1.append(idx)
#             print(valid_indices)
    valid_indices2 = []
    for idx in indices2:
        if idx in idx2token and idx not in (target_vocab['<pad>'], target_vocab['<sos>'], target_vocab['<eos>']):
            if idx < 10:
                valid_indices2.append(idx)
            else:
                valid_indices2.append(idx-3)
#             print(valid_indices)
    decoded_text1 = ''
    decoded_text2 = ''
    l1 = len(valid_indices1)
    val_ind2 = valid_indices2[:l1]
    for idx2 in val_ind2:
#         decoded_text1 += idx2token[idx1]
        decoded_text2 += idx2token[idx2]
    
    return decoded_text2

In [70]:
def pred(model, dataloader, device):
    model.eval()
    predictions = []
    actual = []
    with torch.no_grad():
        for latin, devanagari in dataloader:#, desc='Evaluating', unit='batch'):
            latin = latin.to(device)
            devanagari = devanagari.to(device)
            output = model(latin, devanagari,0)
            deb = devanagari.cpu().numpy()
            actual.append(deb)
            output = output.argmax(2)
            latin = latin.cpu().numpy()
            output = output.cpu().numpy()
            predictions.append((latin, output))
    return predictions, actual


# Make sure to define the reverse dictionaries for converting indices back to text
latin_idx2token = {idx: char for char, idx in input_vocab.items()}
bangla_idx2token = {idx: char for char, idx in target_vocab.items()}

In [71]:
test_predictions, actual = pred(trained_model, test_loader_ben, device)
results = []
for (src_indices, output_indices),act_ind in zip(test_predictions,actual):
#     print(src_indices)
#     print('\njkl',output_indices)
    # Since our data loader might have batch size greater than 1, iterate through each example in the batch
    for i in range(src_indices.shape[0]):
        input_text = decode_indices(src_indices[i], latin_idx2token, input_vocab)
        actual_target_text = decode_indices(act_ind[i], bangla_idx2token, target_vocab)
        predicted_text = decode_indices_target1(act_ind[i],output_indices[i], bangla_idx2token, target_vocab)#decode_indices_target(output_indices[i], bangla_idx2token, target_vocab) 
        results.append([input_text, actual_target_text, predicted_text])
        
        print(f'SL. {i} Input Text: {input_text} -> Actual target: {actual_target_text} -> Predicted Text: {predicted_text}')
#     break

SL. 0 Input Text: kaarentabaahee -> Actual target: কারেন্টবাহী -> Predicted Text: কােনতাবহীী়
SL. 1 Input Text: mashterpiece -> Actual target: মাস্টারপিস -> Predicted Text: মযাশটারপিয
SL. 2 Input Text: cheeken -> Actual target: চিকেন -> Predicted Text: চেকেন
SL. 3 Input Text: ekdaala -> Actual target: একডালা -> Predicted Text: একদালা
SL. 4 Input Text: neerbachokra -> Actual target: নির্বাচকরা -> Predicted Text: নিরবাচকরার
SL. 5 Input Text: neture -> Actual target: নেচার -> Predicted Text: নেতুর
SL. 6 Input Text: michilkey -> Actual target: মিছিলকে -> Predicted Text: মিচিলকে
SL. 7 Input Text: chitfund -> Actual target: চিটফান্ড -> Predicted Text: চিটফুনডড
SL. 8 Input Text: panchanan -> Actual target: পঞ্চানন -> Predicted Text: পঞচনননন
SL. 9 Input Text: manna -> Actual target: মন্ন -> Predicted Text: মননা
SL. 10 Input Text: portillo -> Actual target: পর্টিল্লো -> Predicted Text: পরিলোলোোল
SL. 11 Input Text: quess -> Actual target: কুয়েস -> Predicted Text: কুয়েস
SL. 12 Input Text: budh 

In [72]:
# Writing results to CSV
import csv
with open('results.csv', mode='w', newline='', encoding='utf-8') as file:
    writer = csv.writer(file)
    writer.writerow(['Input Text', 'Actual Target', 'Predicted Text'])
    writer.writerows(results)

In [73]:
df1 = pd.read_csv('results.csv')
df1

,Input Text,Actual Target,Predicted Text
0,kaarentabaahee,কারেন্টবাহী,কােনতাবহীী়
1,mashterpiece,মাস্টারপিস,মযাশটারপিয
2,cheeken,চিকেন,চেকেন
3,ekdaala,একডালা,একদালা
4,neerbachokra,নির্বাচকরা,নিরবাচকরার
...,...,...,...
4090,samanjasyapurno,সামঞ্জস্যপূর্ণ,সমঞজসযপূরণওণণর
4091,fuds,ফুডস,ফুডস
4092,bannar,ব্যানার,বাননারে
4093,songosthao,সংস্থাও,সংসথাও়
